<a href="https://colab.research.google.com/github/vvh4909/hello_world/blob/master/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers flask requests tensorflow

In [3]:
%%writefile chatbot.py

# Step 1: Load the necessary libraries and models
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, pipeline, TFAutoModelForSeq2SeqLM
from flask import Flask, request, jsonify
import json

# Load Sentiment Analysis Model
sentiment_model = TFAutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
sentiment_tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

# Load Summarization Model
summarization_model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
summarization_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

# Load Question Answering Model
qa_model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
qa_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

# Load Generic Decoder-Only Model
pipe = pipeline("text-generation", model="gpt2-large")


# Initialize Flask App
app = Flask(__name__)
conversation_history = []
conversation_summaries = []


def summarize_conversation(conversation):
    conversation_text = " ".join(conversation)
    inputs = summarization_tokenizer(f'Summarize: {conversation_text}', return_tensors="tf", max_length=512, truncation=True, padding=True)
    summary_ids = summarization_model.generate(inputs['input_ids'], max_length=500, num_beams=4, early_stopping=True)
    summary = summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def summarize_text(text):
    inputs = summarization_tokenizer(f'Summarize: {text}', return_tensors="tf", max_length=512, truncation=True, padding=True)
    summary_ids = summarization_model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)
    summary = summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def answer_question(question):
    inputs = qa_tokenizer(f'question: {question} answer: ', return_tensors="tf")
    answer_ids = qa_model.generate(inputs['input_ids'])
    answer = qa_tokenizer.decode(answer_ids[0], skip_special_tokens=True)
    return answer

def generate_response(message):
    # context = " ".join(conversation_history)
    sentiment = analyze_sentiment(message)
    if sentiment == "negative":
        prompt = f"The user is angry. Their message: {message}"
    else:
        prompt = message
    response = pipe(prompt)[0]['generated_text']
    return response

def analyze_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="tf", truncation=True)
    outputs = sentiment_model(inputs)
    sentiment_score = tf.nn.softmax(outputs.logits, axis=-1).numpy()
    sentiment = "positive" if sentiment_score[0][1] > 0.5 else "negative"
    return sentiment


@app.route('/reset', methods=['POST'])
def reset_conversation():
    global conversation_history
    global conversation_summaries
    summary = summarize_conversation(conversation_history)
    conversation_summaries.append(summary)
    conversation_history = []
    return jsonify({"message": "Conversation has been reset.", "summary": summary})

@app.route('/greet', methods=['GET'])
def greet_user():
    return jsonify({"message": "Hello! How can I assist you today?"})

@app.route('/chat', methods=['POST'])
def chat():
    global conversation_history
    user_message = request.json.get('message')
    conversation_history.append(f'User: {user_message}')

    # Determine the task based on the message
    if "summarize" in user_message.lower():
        print("summarize")
        response = summarize_text(user_message)
    elif "?" in user_message:
        print("question")
        response = answer_question(user_message)
    else:
        print("default")
        response = generate_response(user_message)

    sentiment = analyze_sentiment(user_message)
    conversation_history.append(f'Agent: {response}')

    return jsonify({"response": response})


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

Writing chatbot.py


In [4]:
!nohup python chatbot.py &

nohup: appending output to 'nohup.out'


In [5]:
!sudo lsof -i -P -n | grep LISTEN


node         7 root   21u  IPv6  19199      0t0  TCP *:8080 (LISTEN)
kernel_ma   20 root    3u  IPv4  18933      0t0  TCP 172.28.0.12:6000 (LISTEN)
colab-fil   70 root    3u  IPv4  19249      0t0  TCP 127.0.0.1:3453 (LISTEN)
jupyter-n   91 root    7u  IPv4  21522      0t0  TCP 172.28.0.12:9000 (LISTEN)
python3   2584 root   21u  IPv4  93902      0t0  TCP 127.0.0.1:39523 (LISTEN)
python3   2619 root    3u  IPv4  94590      0t0  TCP 127.0.0.1:40851 (LISTEN)
python3   2619 root    4u  IPv4  94591      0t0  TCP 127.0.0.1:47335 (LISTEN)


In [ ]:
from requests import post, get

response = get(f"http://localhost:5000/greet").json()  # or "http://127.0.0.1:5000/greet"
response

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /greet (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79fe19a0db70>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
from requests import post, get
from socket import gethostname, gethostbyname
ip = gethostbyname(gethostname()) # 172.28.0.12
response = get(f"http://localhost:5000/greet").json()
response

ConnectionError: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /greet (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79fe19a0d6f0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [6]:
from requests import post, get
from socket import gethostname, gethostbyname
ip = gethostbyname(gethostname()) # 172.28.0.12
response = get(f"http://{ip}:5000/greet").json()
response

ConnectionError: HTTPConnectionPool(host='172.28.0.12', port=5000): Max retries exceeded with url: /greet (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7cafcbe6b2e0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
response = post(f"http://{ip}:5000/chat", json= {'message': 'What is the capital of France?'}).json()
response

ConnectionError: HTTPConnectionPool(host='172.28.0.12', port=5000): Max retries exceeded with url: /chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79fe19a0f610>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
!sudo lsof -i -P -n | grep LISTEN

node         7 root   21u  IPv6  19269      0t0  TCP *:8080 (LISTEN)
kernel_ma   19 root    3u  IPv4  18959      0t0  TCP 172.28.0.12:6000 (LISTEN)
colab-fil   69 root    3u  IPv4  19326      0t0  TCP 127.0.0.1:3453 (LISTEN)
jupyter-n   86 root    7u  IPv4  20397      0t0  TCP 172.28.0.12:9000 (LISTEN)
python3   4385 root   21u  IPv4 169394      0t0  TCP 127.0.0.1:33335 (LISTEN)
python3   4418 root    3u  IPv4 170374      0t0  TCP 127.0.0.1:40263 (LISTEN)
python3   4418 root    4u  IPv4 170375      0t0  TCP 127.0.0.1:33241 (LISTEN)


In [7]:
!ping 172.28.0.12

/bin/bash: line 1: ping: command not found


In [1]:
# Install necessary packages
!pip install transformers datasets tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset


In [3]:
# Load the SST2 dataset
dataset = load_dataset("stanfordnlp/sst2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [4]:
# Load the DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding='max_length', truncation=True, max_length=128, return_tensors='tf')

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [6]:
# Convert the tokenized dataset to a TensorFlow dataset
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols="label",
    shuffle=True,
    batch_size=64
)

validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols="label",
    shuffle=False,
    batch_size=64
)


In [7]:
for batch in train_dataset.take(1):
    print(batch[0]['input_ids'].shape)
    print(batch[0]['attention_mask'].shape)
    print(batch[1].shape)
    break

(64, 128)
(64, 128)
(64,)


In [8]:

# Load the pre-trained DistilBERT model
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [9]:
# Freeze the DistilBERT layers
model.layers[0].trainable = False

In [10]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 592130 (2.26 MB)
Non-trainable params: 66362880 (253.15 MB)
_________________________________________________________________


In [11]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)


In [ ]:
# Train the model
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=3
)

Epoch 1/3
1053/1053 [==============================] - 301s 275ms/step - loss: 0.4431 - sparse_categorical_accuracy: 0.8003 - val_loss: 0.3885 - val_sparse_categorical_accuracy: 0.8291
Epoch 2/3
 117/1053 [==>...........................] - ETA: 4:25 - loss: 0.3813 - sparse_categorical_accuracy: 0.8312

In [1]:
# Save the model
model.save_pretrained("./distilbert-sst2")

NameError: name 'model' is not defined